In [1]:
import numpy as np
import pandas as pd
pd.set_option("display.max_columns", 500)
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls

from sklearn.preprocessing import LabelEncoder
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.model_selection import TimeSeriesSplit, cross_val_score
from sklearn.metrics import roc_auc_score
from hyperopt import fmin, hp, tpe, space_eval

from sklearn.model_selection import KFold, TimeSeriesSplit
import lightgbm as lgb
from time import time
from tqdm import tqdm_notebook

from xgboost import XGBClassifier
import os

import gc
import warnings
warnings.filterwarnings('ignore')

/Users/didi/anaconda3/lib/python3.6/site-packages/lightgbm/__init__.py:46: UserWarning:

Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.1) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.



In [2]:
NROWS = None
features = ["TransactionID", "P_emaildomain", "card1", "card2", "card3",\
           "card4", "card5", "card6", "addr1", "addr2", "TransactionAmt"]
test = pd.read_csv('../temp/test_temp' + str(NROWS) + '.csv', usecols=features)

In [3]:
pre = pd.read_csv('../output/final_sub_v2.csv')

In [4]:
print(pre.shape)
pre.head()

(506691, 2)


,TransactionID,isFraud
0,3663549,0.038136
1,3663550,0.072050
2,3663551,0.069657
3,3663552,0.091089
4,3663553,0.042804


In [5]:
# test['TransactionAmt_card_addr1_P_emaildomain'] = test["TransactionAmt"].apply(lambda x: str(x)) + "_" +\
#                 test["card1"].apply(lambda x: str(x)) + "_" + test["card2"].apply(lambda x: str(x)) +\
#                 "_" + test["card3"].apply(lambda x: str(x)) + "_" + test["card4"].apply(lambda x: str(x)) +\
#                 "_" + test["card5"].apply(lambda x: str(x)) + "_" + test["card6"].apply(lambda x: str(x)) +\
#                 "_" + test["addr1"].apply(lambda x: str(x)) + "_" + test["P_emaildomain"].apply(lambda x: str(x))

# test['TransactionAmt_card_addr1'] = test["TransactionAmt"].apply(lambda x: str(x)) + "_" +\
#                 test["card1"].apply(lambda x: str(x)) + "_" + test["card2"].apply(lambda x: str(x)) +\
#                 "_" + test["card3"].apply(lambda x: str(x)) + "_" + test["card4"].apply(lambda x: str(x)) +\
#                 "_" + test["card5"].apply(lambda x: str(x)) + "_" + test["card6"].apply(lambda x: str(x)) +\
#                 "_" + test["addr1"].apply(lambda x: str(x))

test['TransactionAmt_card'] = test["TransactionAmt"].apply(lambda x: str(x)) + "_" +\
                test["card1"].apply(lambda x: str(x)) + "_" + test["card2"].apply(lambda x: str(x)) +\
                "_" + test["card3"].apply(lambda x: str(x)) + "_" + test["card4"].apply(lambda x: str(x)) +\
                "_" + test["card5"].apply(lambda x: str(x)) + "_" + test["card6"].apply(lambda x: str(x))

In [6]:
print(test.shape)
test.head()

(506691, 12)


,TransactionID,TransactionAmt,card1,card2,card3,card4,card5,card6,addr1,addr2,P_emaildomain,TransactionAmt_card
0,3663549,31.95,10409,111.0,150.0,visa,226.0,debit,170.0,87.0,gmail.com,31.95_10409_111.0_150.0_visa_226.0_debit
1,3663550,49.00,4272,111.0,150.0,visa,226.0,debit,299.0,87.0,aol.com,49.0_4272_111.0_150.0_visa_226.0_debit
2,3663551,171.00,4476,574.0,150.0,visa,226.0,debit,472.0,87.0,hotmail.com,171.0_4476_574.0_150.0_visa_226.0_debit
3,3663552,284.95,10989,360.0,150.0,visa,166.0,debit,205.0,87.0,gmail.com,284.95_10989_360.0_150.0_visa_166.0_debit
4,3663553,67.95,18018,452.0,150.0,mastercard,117.0,debit,264.0,87.0,gmail.com,67.95_18018_452.0_150.0_mastercard_117.0_debit


In [7]:
pre = pre.merge(test, on="TransactionID", how="left")

In [8]:
pre[["TransactionID", "isFraud", "TransactionAmt_card"]]

,TransactionID,isFraud,TransactionAmt_card
0,3663549,0.038136,31.95_10409_111.0_150.0_visa_226.0_debit
1,3663550,0.072050,49.0_4272_111.0_150.0_visa_226.0_debit
2,3663551,0.069657,171.0_4476_574.0_150.0_visa_226.0_debit
3,3663552,0.091089,284.95_10989_360.0_150.0_visa_166.0_debit
4,3663553,0.042804,67.95_18018_452.0_150.0_mastercard_117.0_debit
5,3663554,0.110046,57.95_12839_321.0_150.0_visa_226.0_debit
6,3663555,0.177801,87.0_16560_476.0_150.0_visa_126.0_debit
7,3663556,0.194189,390.0_15066_170.0_150.0_mastercard_102.0_credit
8,3663557,0.002560,103.95_2803_100.0_150.0_visa_226.0_debit
9,3663558,0.133543,117.0_12544_321.0_150.0_visa_226.0_debit


- 可疑的TransactionID
3663564
3663572

In [32]:
val = pre.loc[pre["TransactionID"] == 3663564, "TransactionAmt_card"]
val

15    57.95_7826_481.0_150.0_mastercard_224.0_debit
Name: TransactionAmt_card, dtype: object

In [36]:
pre.loc[pre["TransactionAmt_card"] == "57.95_7826_481.0_150.0_mastercard_224.0_debit"]

,TransactionID,isFraud,TransactionAmt,card1,card2,card3,card4,card5,card6,addr1,addr2,P_emaildomain,TransactionAmt_card
15,3663564,0.121796,57.95,7826,481.0,150.0,mastercard,224.0,debit,325.0,87.0,gmail.com,57.95_7826_481.0_150.0_mastercard_224.0_debit
4644,3668193,0.056841,57.95,7826,481.0,150.0,mastercard,224.0,debit,325.0,87.0,yahoo.com,57.95_7826_481.0_150.0_mastercard_224.0_debit
5586,3669135,0.096183,57.95,7826,481.0,150.0,mastercard,224.0,debit,387.0,87.0,yahoo.com,57.95_7826_481.0_150.0_mastercard_224.0_debit
5644,3669193,0.101672,57.95,7826,481.0,150.0,mastercard,224.0,debit,387.0,87.0,yahoo.com,57.95_7826_481.0_150.0_mastercard_224.0_debit
6680,3670229,0.024275,57.95,7826,481.0,150.0,mastercard,224.0,debit,184.0,87.0,gmail.com,57.95_7826_481.0_150.0_mastercard_224.0_debit
10711,3674260,0.086555,57.95,7826,481.0,150.0,mastercard,224.0,debit,325.0,87.0,hotmail.com,57.95_7826_481.0_150.0_mastercard_224.0_debit
11490,3675039,0.049193,57.95,7826,481.0,150.0,mastercard,224.0,debit,387.0,87.0,NaN,57.95_7826_481.0_150.0_mastercard_224.0_debit
12370,3675919,0.151629,57.95,7826,481.0,150.0,mastercard,224.0,debit,494.0,87.0,me.com,57.95_7826_481.0_150.0_mastercard_224.0_debit
31647,3695196,0.150990,57.95,7826,481.0,150.0,mastercard,224.0,debit,325.0,87.0,yahoo.com,57.95_7826_481.0_150.0_mastercard_224.0_debit
41671,3705220,0.075382,57.95,7826,481.0,150.0,mastercard,224.0,debit,325.0,87.0,gmail.com,57.95_7826_481.0_150.0_mastercard_224.0_debit


In [37]:
pre.loc[(pre["TransactionAmt_card"] == "57.95_7826_481.0_150.0_mastercard_224.0_debit") &\
       (pre["isFraud"] > 0.5)]

,TransactionID,isFraud,TransactionAmt,card1,card2,card3,card4,card5,card6,addr1,addr2,P_emaildomain,TransactionAmt_card


In [48]:
sub1 = pd.read_csv('../output/final_sub_v2.csv')

In [49]:
TransactionID = 3668193

sub1.loc[sub1['TransactionID'] == TransactionID, "isFraud"]

4644    0.056841
Name: isFraud, dtype: float64

In [50]:
sub1.loc[sub1['TransactionID'] == TransactionID, "isFraud"] = 0.

In [51]:
sub1.loc[sub1['TransactionID'] == TransactionID, "isFraud"]

4644    0.0
Name: isFraud, dtype: float64

In [52]:
sub1.to_csv('../output/final_sub_v2_' + str(TransactionID) + '_0.csv', index=False)